<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/Z%C4%B0YA_16_CO%C4%B0N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Gerekli kütüphaneleri yükle ve içe aktar
!pip install yfinance pandas_ta matplotlib --quiet
import pandas as pd
import yfinance as yf
import pandas_ta as ta
from datetime import datetime

# 1. Ayarlar ve Konfigürasyon
class Config:
    SYMBOL_GROUPS = {
        'CRYPTO': [
            'BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD',
            'ADA-USD', 'SOL-USD', 'DOGE-USD', 'DOT-USD',
            'AVAX-USD', 'SHIB-USD', 'MATIC-USD', 'LTC-USD',
            'UNI-USD', 'LINK-USD', 'ATOM-USD', 'XLM-USD'
        ]
    }
    TIME_INTERVAL = '1h'  # 1 saatlik veri
    PERIOD = '30d'        # 30 günlük veri
    INDICATORS = {
        'rsi': {'length': 14},
        'macd': {'fast': 12, 'slow': 26, 'signal': 9},
        'bbands': {'length': 20, 'std': 2}
    }

# 2. Geliştirilmiş Veri Yönetim Sistemi
class DataManager:
    @staticmethod
    def get_clean_data(symbol):
        try:
            df = yf.download(
                tickers=symbol,
                period=Config.PERIOD,
                interval=Config.TIME_INTERVAL,
                progress=False,
                group_by='ticker',
                timeout=10  # Zaman aşımı süresi
            )

            # Çoklu sütun kontrolü
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(0)

            # Sütun varlığını kontrol et
            if 'Close' not in df.columns:
                print(f"⚠️ {symbol} için Close sütunu eksik")
                return None

            # Veri temizleme
            df = df[['Close']].dropna().ffill().bfill()

            if len(df) < 50:  # Minimum 50 veri noktası
                print(f"⚠️ Yetersiz veri: {symbol} ({len(df)}/50)")
                return None
            return df
        except Exception as e:
            print(f"❌ {symbol} hatası: {str(e)}")
            return None

# 3. Teknik Analiz Motoru
class TechnicalAnalyzer:
    @staticmethod
    def calculate_technicals(df):
        try:
            df.ta.rsi(**Config.INDICATORS['rsi'], append=True)
            df.ta.macd(**Config.INDICATORS['macd'], append=True)
            df.ta.bbands(**Config.INDICATORS['bbands'], append=True)
            return df.iloc[-1].to_dict()
        except Exception as e:
            print(f"📉 {symbol} gösterge hatası: {str(e)}")
            return None

# 4. Sonuçları İşleme
def analyze_coins():
    results = []
    for symbol in Config.SYMBOL_GROUPS['CRYPTO']:
        print(f"\n🔍 Analiz: {symbol}")
        df = DataManager.get_clean_data(symbol)
        if df is not None:
            data = TechnicalAnalyzer.calculate_technicals(df)
            if data:
                results.append({
                    'Coin': symbol.replace('-USD', ''),
                    'Fiyat ($)': data['Close'],
                    'RSI (14)': data.get('RSI_14', None),
                    'MACD': data.get('MACD_12_26_9', None),
                    'BB Üst': data.get('BBU_20_2.0', None),
                    'BB Alt': data.get('BBL_20_2.0', None)
                })
    return pd.DataFrame(results)

# 5. Çalıştırma ve Sonuç
print("🔄 Veriler çekiliyor ve analiz ediliyor...")
results_df = analyze_coins()
print(f"\n📅 Son Güncelleme: {datetime.now().strftime('%d %B %Y %H:%M')}")
print(f"📈 Analiz Edilen Coin Sayısı: {len(results_df)}/{len(Config.SYMBOL_GROUPS['CRYPTO'])}")

if not results_df.empty:
    print("\n✅ Sonuçlar:")
    # Formatlama işlemi
    formatted_df = results_df.style.format({
        'Fiyat ($)': '{:.2f}',
        'RSI (14)': '{:.2f}',
        'MACD': '{:.2f}',
        'BB Üst': '{:.2f}',
        'BB Alt': '{:.2f}'
    }, na_rep='N/A')  # NaN değerler için "N/A" göster
    display(formatted_df)
else:
    print("\n❗ Hiçbir coin için veri bulunamadı!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
🔄 Veriler çekiliyor ve analiz ediliyor...

🔍 Analiz: BTC-USD

🔍 Analiz: ETH-USD

🔍 Analiz: BNB-USD

🔍 Analiz: XRP-USD

🔍 Analiz: ADA-USD

🔍 Analiz: SOL-USD

🔍 Analiz: DOGE-USD

🔍 Analiz: DOT-USD

🔍 Analiz: AVAX-USD

🔍 Analiz: SHIB-USD

🔍 Analiz: MATIC-USD

🔍 Analiz: LTC-USD

🔍 Analiz: UNI-USD

🔍 Analiz: LINK-USD

🔍 Analiz: ATOM-USD

🔍 Analiz: XLM-USD

📅 Son Güncelleme: 04 February 2025 08:05
📈 Analiz Edilen Coin Sayısı: 16/16

✅ Sonuçlar:


,Coin,Fiyat ($),RSI (14),MACD,BB Üst,BB Alt
0,BTC,98329.72,45.32,430.38,103426.17,96241.79
1,ETH,2694.16,43.60,-6.41,2913.22,2589.78
2,BNB,567.13,34.64,-5.30,628.17,562.93
3,XRP,2.48,42.05,0.00,2.87,2.38
4,ADA,0.72,38.02,-0.01,0.84,0.70
5,SOL,203.74,44.21,0.74,223.55,196.83
6,DOGE,0.26,41.75,-0.00,0.30,0.25
7,DOT,4.64,38.49,-0.06,5.36,4.56
8,AVAX,25.63,39.06,-0.24,29.31,25.09
9,SHIB,0.00,41.97,0.00,0.00,0.00
